### ThreadsLocking can guarantee both visibility and atomicity; volatile variables can
only guarantee visibility.
There are two primary ways to create a thread. 1) Extend the `Thread` class and override the run method.

In [ ]:
public class Worker extends Thread{
    @Override
    public void run(){
        System.out.println("Worker thread running");
    }
}

Worker worker = new Worker();

2) The preferred way is to pass a `Runnable` instance to Thread constructor

In [ ]:
Runnable runnable = new Runnable{
    public void run(){
        System.out.println("Worker thread running");
    }
}

Thread worker = new Thread(runnable);

**Starting a thread:** use the `start` method of the Thread class. The `isAlive` method can tell if the thread is currently running or not. isAlive returns false if the thread has not been started or the thread has been terminated.  

**Stopping a thread:** is not as straight forward as starting it. The only way to terminate a thread is to arrange for its run method to complete. The thread class has a `stop` method, but it has been deprecated. Thread throwing an unchecked exception can also terminate the thread. Two approaches for stopping a thread: 
1) setting a flag

In [ ]:
public class Worker extends Thread{
    private volatile boolean done = false;
    
    @Override
    public void run(){
        while(!done){
            // Do something
        }
    }
    
    // Other methods
}

An external code can stop this thread by setting done as true. But the thread will only stop when the execution reaches the top of the loop (and not immediately).  
2) Using interrupts: we can call `interrupt` method of any thread. It has two effects. First, it sets internal interrupted flag of the thread to true. Second, it causes any blocking method like `sleep`, `join`, `wait` to stop blocking and throw `InterruptedException`.

In [ ]:
public class Worker extends Thread{
    
    @Override
    public void run(){
        while(!isInterrupted()){
            // Do something
        }
    }
    
    // Other methods
}

Worker worker = new Worker();
worker.start();
worker.interrupt();

**Pausing a thread:** the Thread class has `suspend` and `resume` methods which have been deprecated. It is still possible for a thread to suspend its own execution for a specified amount of time using `sleep` method. The least amount of time a thread can sleep on most Java implementations is 20 to 50 ms.

In [6]:
public class DelayedRandomGenerator extends Thread {
    public void run() {
        Random random = new Random();
        while (!isInterrupted()) {
            System.out.print(random.nextInt() + "\t");
            try {
                sleep(200);
            } catch (InterruptedException e) {
                break;
            }
        }
    }
}

DelayedRandomGenerator generator = new DelayedRandomGenerator();
generator.start();

try {
    Thread.sleep(1000);
} catch (InterruptedException e) {
    System.exit(0);
}        

generator.interrupt();

1152488192	2140810343	1285862342	1650368227	-1184878083	1039227028	278517272	-1543200418	401414008	-531716027	

### Race Condition
When two or more threads access a given state variable, and try to change it, it can lead to a race condition.

In [ ]:
class SharedUnsafeObject {
    private long counter = 0;

    public void incrementCounter() {
        this.counter++;
    }

    public long getCounter() {
        return this.counter;
    }
}

SharedUnsafeObject obj = new SharedUnsafeObject();

// 5 threads trying to access counter variable of the
// same object
for (int i = 0; i < 5; i++) {
    new Thread(() -> {
        obj.incrementCounter();
    }).start();
}

try {
    Thread.sleep(1000);
    System.out.println(obj.getCounter()); // Output not guaranteed as 5
} catch (InterruptedException e) {
    e.printStackTrace();
}

Race condition in above case happens because the operation `this.counter++;` is not atomic. It is comprised of few substeps and different threads can interleave execution of these steps. Another race condition is present in the below version of singleton:

In [ ]:
public class Singleton{
    private static Singleton instance = null;
    
    public Singleton getInstance(){
        if(instance == null){
            instance = new Singleton();
        }
        
        return instance;
    }
    
    // Private constructors, etc
}

Calling `getInstance` can lead to creation of two different objects. One way to prevent race condition is to utilise *atomic operations*. The increment operation in previous example is a *read-modify-write* operation, whereas operation in singleton example is *check then act* operation. Both of these must be atomic. The read-modify-write operation problem can be solved using an *Atomic Type*.

In [ ]:
class SharedSafeObject {
    private AtomicLong counter = new AtomicLong(0);

    public void incrementCounter() {
        this.counter.incrementAndGet();
    }

    public long getCounter() {
        return this.counter.get();
    }
}

Another solution is to use `synchronized` keyword. A method marked as synchronized can be executed by a single thread at any given moment. Whenever a thread wants to execute a synchronized method, it first obtain the corresponding lock. If the lock is held by another thread, then the thread requesting lock must wait till the lock is released.  

Below is the synchronized version of the shared object example used earlier:

In [ ]:
class SharedObject {
    private long counter = 0;

    public synchronized void incrementCounter() {
        this.counter++;
    }

    public synchronized long getCounter() {
        return this.counter;
    }
}

The above example is equivalent to:

In [ ]:
// The lock object is 'this'
class SharedObject {
    private long counter = 0;

    public void incrementCounter() {
        synchronized(this){
            this.counter++;
        }
    }

    public long getCounter() {
        synchronized(this){
            return this.counter;
        }
    }
}

If an object has two independent state variables, it better to use to different lock objects such that two different threads can access two states concurrently.

In [ ]:
class SharedObject {
    private long c1 = 0;
    private long c2 = 0;
    private Object o1 = new Object();
    private Object o2 = new Object();

    public void incrementC1() {
        synchronized(o1){
            this.c1++;
        }
    }

    public void incrementC2() {
        synchronized(o2){
            this.c2++;
        }
    }
}

### Visibility
Synchronization also has another significant, and subtle, aspect: memory visibility. We want not only to prevent one thread from modifying the state of an object when another is using it, but also to ensure that when a thread modifies the state of an object, other threads can actually see the changes that were made.  

In general, there is no guarantee that the reading thread will see a value written by another thread on a timely basis, or even at all.

In [ ]:
public class NoVisibility {
    private static boolean ready;
    private static int number;
    
    private static class ReaderThread extends Thread {
        public void run() {
            while (!ready)
                // Yield means the current thread is willing to relinquish its current use of processor
                // but it'd like to be scheduled back soon as possible. 
                Thread.yield();
            System.out.println(number);
        }
    }
    
    public static void main(String[] args) {
        new ReaderThread().start();
        number = 42;
        ready = true;
    }
}

The problem with above code is that it is not necessary that the updated values of number or ready (made by main thread), is visible to the ReaderThread. The ReaderThread may print 42, or the loop may run indefinitely.  

One reason for reading stale values is the cache of the CPU cores. Each core of modern CPUs has its own cache. So if the reading and writing thread runs on different cores the reading thread sees a cached value and not the value written by the writing thread.

The ReaderThread may also output 0. This can be due to *instruction reordering*. JVM may reorder two instructions which are not related to each other. For example:
```
a = b + c
d = a + e

l = m + n
y = x + z
```
can be reordered to:
```
a = b + c

l = m + n
y = x + z

d = a + e
```

It is not sufficient to synchronize just the setter. The getter must also be synchronized, else we may get stale value.

In [ ]:
public class SynchronizedInteger {
    private int value;
    
    public synchronized int get() { 
        return value; 
    }
    
    public synchronized void set(int value) { 
        this.value = value; 
    }
}

Unlike primitives like int, float and double are not read from memory in one atomic step. Rather, 32 bits are read at a time. So it may be possible to read 32 bits of fresh value and 32 bits of stale value.

**Volatile variables:** are another solution to visibility problem. Any read or write on volatile variables are done to the main memory rather than CPU caches. The visibility effects of volatile variables extend beyond the value of the volatile variable itself. When thread A writes to a volatile variable and subsequently thread B reads that same variable, the values of all variables that were visible to A prior to writing to the volatile variable become visible to B after reading the volatile variable.

The above example of SynchronizedInteger can modified to:

In [ ]:
public class VolatileInteger {
    private volatile int value;
    
    public int get() { 
        return value; 
    }
    
    public void set(int value) { 
        this.value = value; 
    }
}

Locking using synchronized blocks can guarantee both visibility and atomicity; volatile variables can only guarantee visibility.